## Klassifikation der einzelnen Objecte im mvtec_anomaly_detection Dataset 

### Importieren der Bibliotheken und Files 

In [1]:
from read_and_label_Images_only_objects import read_and_label_Images_only_objects
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


In [2]:
%load_ext autoreload
%autoreload 2